This is playing with pytorch framework for EHR modeling. In general, a patient's health record can be represented as a sequence of visits. Each visit has certain features, and can be represented as a list of medical codes.

For simplicity, we are starting with the data structure that a patient's health record is a list of list, following the line of work from Jimeng Sun's lab. We will use codes from Ed Choi to manipulate the data. 

The core model is an RNN , either LSTM, GRU or Vanilla RNN.

# todos:
* None for now

In [1]:
%matplotlib inline
from __future__ import print_function, division
from io import open
import string
import re
import random
import sklearn 
from sklearn.metrics import roc_auc_score
import plotly.plotly as py 
import plotly.graph_objs as go
import torch
import torch.nn as nn
import torch.autograd as autograd
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
import torch.utils.data as Data

use_cuda = torch.cuda.is_available()
torch.cuda.set_device(1)

import sys, random
import numpy as np
try:
    import cPickle as pickle
except:
    import pickle

from torchviz import make_dot, make_dot_from_trace

# for windows only    
import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'
#os.environ["CUDA_DEVICE_ORDER"]='PCI_BUS_ID'
#os.environ["CUDA_VISIBLE_DEVICES"]='0'

import bnlstm

In [2]:
#%env CUDA_DEVICE_ORDER=PCI_BUS_ID
#%env CUDA_VISIBLE_DEVICES=1
use_cuda

True

In [3]:
train_sl = pickle.load(open('pdata_3hosp/h143_train', 'rb'), encoding='bytes')
valid_sl = pickle.load(open('pdata_3hosp/h143_valid', 'rb'), encoding='bytes')
test_sl =  pickle.load(open('pdata_3hosp/h143_test', 'rb'), encoding='bytes')
#test_sh_143 = pickle.load(open('pdata_3hosp/h143_test_sh', 'rb'), encoding='bytes')
#test_l_143 = pickle.load(open('pdata_3hosp/h143_test_lng', 'rb'), encoding='bytes')
print (len(train_sl),len(valid_sl),len(test_sl))

29912 4272 8545


In [4]:
test_118 = pickle.load(open('pdata_3hosp/h118_m', 'rb'), encoding='bytes')
test_104 = pickle.load(open('pdata_3hosp/h104_m', 'rb'), encoding='bytes')
test_3hosp = pickle.load(open('pdata_3hosp/h3hs_m', 'rb'), encoding='bytes')
#test_sh_118 = pickle.load(open('pdata_3hosp/h118_test_sh', 'rb'), encoding='bytes')
#test_l_118 = pickle.load(open('pdata_3hosp/h118_test_lng', 'rb'), encoding='bytes')
#test_sh_104 = pickle.load(open('pdata_3hosp/h104_test_sh', 'rb'), encoding='bytes')
#test_l_104 = pickle.load(open('pdata_3hosp/h104_test_lng', 'rb'), encoding='bytes')
#test_sh_3hosp = pickle.load(open('pdata_3hosp/h3hs_test_sh', 'rb'), encoding='bytes')
#test_l_3hosp = pickle.load(open('pdata_3hosp/h3hs_test_lng', 'rb'), encoding='bytes')

In [5]:
'''
from collections import OrderedDict
def load_params(model):
    params = OrderedDict()
    weights = np.load(model)
    for k,v in weights.iteritems():
        params[k] = v
    return params

param= load_params("/data/projects/cerner_dev/cl2_hosp_retain_out/h50_cl2_all_hosp_143_cl2m.out.0.npz")
emb_pretrain = param['W_emb']
'''

'\nfrom collections import OrderedDict\ndef load_params(model):\n    params = OrderedDict()\n    weights = np.load(model)\n    for k,v in weights.iteritems():\n        params[k] = v\n    return params\n\nparam= load_params("/data/projects/cerner_dev/cl2_hosp_retain_out/h50_cl2_all_hosp_143_cl2m.out.0.npz")\nemb_pretrain = param[\'W_emb\']\n'

In [6]:
class EHR_RNN(nn.Module):
    def __init__(self, input_size,embed_dim, hidden_size, n_layers=1,dropout_r=0.1,cell_type='BNLSTM',bi=False , preTrainEmb=''):
        super(EHR_RNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embed_dim = embed_dim
        self.dropout_r = dropout_r
        self.cell_type = cell_type
        self.preTrainEmb=preTrainEmb
        if bi: self.bi=2 
        else: self.bi=1
              
        if len(self.preTrainEmb)>0:
            emb_t= torch.FloatTensor(np.asmatrix(self.preTrainEmb))
            self.embed= nn.Embedding.from_pretrained(emb_t)#,freeze=False)  
        else:
            self.embed= nn.Embedding(input_size, self.embed_dim,padding_idx=0)
        
        if self.cell_type == "GRU":
            cell = nn.GRU
        elif self.cell_type == "RNN":
            cell = nn.RNN
        elif self.cell_type == "LSTM":
            cell = nn.LSTM
        elif self.cell_type == "BNLSTM":
            cell = bnlstm.LSTM    
        else:
            raise NotImplementedError
      
        if self.cell_type == "BNLSTM":
            self.rnn_c = bnlstm.LSTM (bnlstm.BNLSTMCell, self.embed_dim, hidden_size,num_layers=n_layers,use_bias=False, batch_first=True,dropout= dropout_r,max_length=30)
        else:
            self.rnn_c = cell(self.embed_dim, hidden_size,num_layers=n_layers, dropout= dropout_r , bidirectional=bi , batch_first=True )
        
        self.out = nn.Linear(self.hidden_size*self.bi,1)
        self.sigmoid = nn.Sigmoid()

        
    def EmbedPatient_MB(self, input): # x is a ehr_seq_tensor
        
        mb=[]
        lbt=[]
        seq_l=[]
        self.bsize=len(input)
        lp= len(max(input, key=lambda xmb: len(xmb[1]))[1])
        self.max_len_bn=lp
        #print (lp)
        llv=0
        for x in input:
            lv= len(max(x[1], key=lambda xmb: len(xmb)))
            if llv< lv:
                llv=lv
        #print (llv)
        for pt in input:
            label, ehr_seq_l = pt
            lpx=len(ehr_seq_l)
            seq_l.append(lpx)
            label_tensor = Variable(torch.FloatTensor([[float(label)]]))
            if use_cuda:
                label_tensor = label_tensor.cuda()
            lbt.append(label_tensor)
            ml=(len(max(ehr_seq_l, key=len)))
            ehr_seq_tl=[]
            for ehr_seq in ehr_seq_l: 
                pd=(0, llv-len(ehr_seq))
                result = F.pad(torch.from_numpy(np.asarray(ehr_seq,dtype=int)).type(torch.cuda.LongTensor),pd,"constant", 0)
                if use_cuda:
                    result.cuda()
                ehr_seq_tl.append(result)
            ehr_seq_t= Variable(torch.stack(ehr_seq_tl,0))     
            lpp= lp-lpx
            zp= nn.ZeroPad2d((0,0,lpp,0))
            ehr_seq_t= zp(ehr_seq_t)
            mb.append(ehr_seq_t)
                
        mb_t= Variable(torch.stack(mb,0)) 
        if use_cuda:
            mb_t.cuda()
        embedded = self.embed(mb_t)
        embedded = torch.sum(embedded, dim=2) 
        lbt_t= Variable(torch.stack(lbt,0)) 
        return embedded, lbt_t,seq_l
    
    def init_hidden(self):
        
        h_0 = Variable(torch.rand(self.n_layers*self.bi,self.bsize, self.hidden_size))
        
        if self.cell_type == "LSTM":
            result = (h_0,h_0)
        else: 
            result = h_0
            
        if use_cuda:
            return result.cuda()
        else:
            return result
    
    def forward(self, input):
        
        x_in , lt ,x_lens = self.EmbedPatient_MB(input)
        #x_inp = nn.utils.rnn.pack_padded_sequence(x_in,x_lens,batch_first=True)
        #h_0 = self.init_hidden()
        output, hidden = self.rnn_c(x_in)#,h_0) 
        if self.cell_type == "LSTM":
            hidden=hidden[0]
        if self.bi==2:
            output = self.sigmoid(self.out(torch.cat((hidden[-2],hidden[-1]),1)))
        else:
            output = self.sigmoid(self.out(hidden[-1]))
        return output.squeeze(), lt.squeeze()

# GRU Epoch 2: Valid_auc :  0.8323451289474217  ,& Test_auc :  0.8432979973495972 with Random hidden
## bestValidAuc 0.821507 has a TestAuc of 0.837944 at epoch 2 with default h_0
## with adagrad 0.8213349966422361  ,& Test_auc :  0.8434773468886594 
#LSTM bestValidAuc 0.821653 has a TestAuc of 0.838208 at epoch 7 
 

In [7]:
emb_pretrain=''                       
model = EHR_RNN(input_size=15816, hidden_size=64 ,embed_dim=256, dropout_r=0, cell_type='BNLSTM',bi=False , n_layers=1, preTrainEmb=emb_pretrain)
bmodel_pth = './best_model_BNLSTM.pth'
bmodel_st = './best_model_BNLSTM.st'


if use_cuda:
    model = model.cuda()

/home/lgindybekhet/test/Pytorch_Code_tst/bnlstm.py:172: UserWarning:

nn.init.orthogonal is now deprecated in favor of nn.init.orthogonal_.

/home/lgindybekhet/test/Pytorch_Code_tst/bnlstm.py:179: UserWarning:

nn.init.constant is now deprecated in favor of nn.init.constant_.



In [8]:
def train (tmodel,mini_batch, criterion, optimizer):  
    tmodel.train()
    model.zero_grad()
    output , label_tensor = tmodel(mini_batch)
    loss = criterion(output, label_tensor)
    loss.backward()
    optimizer.step()
   
    return output, loss.item()

In [9]:
# training all samples in random order
import time
import math

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)





In [10]:
def run_model_train(tmodel,dataset,batch_size,learning_rate = 0.05, l2=3e-04,epsl=1e-06 ):
    
    #optimizer = optim.SGD(model.parameters(), lr=learning_rate)#, weight_decay=l2)
    #optimizer = optim.Adadelta(model.parameters(), lr=learning_rate, weight_decay=l2)
    #optimizer = optim.ASGD(model.parameters(), lr=learning_rate, weight_decay=l2 )
    #optimizer = optim.SparseAdam (model.parameters(),lr=learning_rate) #'''lr=learning_rate,''' 
    optimizer = optim.Adagrad (tmodel.parameters(),lr=learning_rate, weight_decay=l2) #'''lr=learning_rate,''' 
    #optimizer = optim.Adamax(filter(lambda p: p.requires_grad, model.parameters()), lr=learning_rate, weight_decay=l2 ,eps=epsl) ### Beta defaults (0.9, 0.999)
    #optimizer = optim.RMSprop (model.parameters(),lr=learning_rate, weight_decay=l2 ,eps=epsl)
    #optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=learning_rate)
    dataset.sort(key=lambda pt:len(pt[1]),reverse=True) 
    # Keep track of losses for plotting
    current_loss = 0
    all_losses = []
    print_every = 10#int(batch_size/2)
    plot_every = 5
    iter=0
    n_batches = int(np.ceil(int(len(dataset)) / int(batch_size)))
    #print('number of Batches',n_batches)
    start = time.time()

    for index in random.sample(range(n_batches), n_batches):
            batch = dataset[index*batch_size:(index+1)*batch_size]
            output, loss = train(tmodel,batch, criterion = nn.BCELoss(), optimizer = optimizer)
            current_loss += loss
            iter +=1
            # Print iter number, loss, name and guess
            #if iter % print_every == 0:
               #print('%d %d%% (%s) %.4f ' % ( iter, iter/ n_batches * 100, timeSince(start), loss))

            # Add current loss avg to list of losses
            if iter % plot_every == 0:
                all_losses.append(current_loss / plot_every)
                current_loss = 0
                
    return current_loss,all_losses


In [11]:
def calculate_auc(test_model, dataset, batch_size=200):

    test_model.eval()
    dataset.sort(key=lambda pt:len(pt[1]),reverse=True) 
    n_batches = int(np.ceil(int(len(dataset)) / int(batch_size)))
    labelVec =[]
    y_hat= []
    
    for index in range(n_batches):
            batch = dataset[index*batch_size:(index+1)*batch_size]
            output, label_t = test_model(batch)
            y_hat.extend(output.cpu().data.view(-1).numpy())
            labelVec.extend(label_t.cpu().data.view(-1).numpy())
    auc = roc_auc_score(labelVec, y_hat)
    
    return auc

In [12]:
epochs=100
batch_size=100
current_loss_l=[]
all_losses_l=[]
train_auc_allep =[]
valid_auc_allep =[]
test_auc_allep=[]
#tests_auc_allep=[]
#testl_auc_allep=[]
bestValidAuc = 0.0
bestTestAuc = 0.0
bestValidEpoch = 0

for ep in range(epochs):
    
    #print (model.embed.weight.data[135] )
    start = time.time()
    current_loss_la,all_losses_la = run_model_train(model,train_sl,batch_size)
    train_time = timeSince(start)
    eval_start = time.time()
    train_auc = calculate_auc(model,train_sl,batch_size)
    test_auc = calculate_auc(model,test_sl,batch_size)
    #test_sh_auc = calculate_auc(model,test_sh_143,batch_size)
    #test_l_auc = calculate_auc(model,test_l_143,batch_size)
    valid_auc = calculate_auc(model,valid_sl,batch_size)
    eval_time = timeSince(eval_start)
    all_losses_l.append (all_losses_la)
    avg_loss = np.mean(all_losses_la)
    train_auc_allep.append(train_auc)
    valid_auc_allep.append(valid_auc)
    test_auc_allep.append(test_auc)
    #testl_auc_allep.append(test_l_auc)
    #tests_auc_allep.append(test_sh_auc)
    current_loss_l.append(current_loss_la)
    print ("Epoch ", ep," Train_auc :", train_auc, " , Valid_auc : ", valid_auc, " ,& Test_auc : " , test_auc," Avg Loss: ", avg_loss, 'Train Time (%s) Eval Time (%s)'%(train_time,eval_time) )
    #print ("Epoch ", ep," Train_auc :", train_auc, " , Valid_auc : ", valid_auc, " ,& Test_auc : " , test_auc,"less than 5 visits history :",test_sh_auc," for more :",test_l_auc ," Avg Loss: ", avg_loss, 'Train Time (%s) Eval Time (%s)'%(train_time,eval_time) )
    
    if valid_auc > bestValidAuc: 
        bestValidAuc = valid_auc
        bestValidEpoch = ep
        bestTestAuc = test_auc
        best_model = model
        torch.save(best_model, bmodel_pth)
        torch.save(best_model.state_dict(), bmodel_st)
    if ep - bestValidEpoch >12: break
            
print ('bestValidAuc %f has a TestAuc of %f at epoch %d ' % (bestValidAuc, bestTestAuc, bestValidEpoch))
torch.save(best_model, bmodel_pth)

# BiGRU:bestValidAuc 0.779129 has a TestAuc of 0.794677 at epoch 8 
#Test Auc for H104:  0.68506300670682 , H118:  0.6706375112328244  , combining together with H143 testset: 0.6858711757128325

# GRU - ASGD  
# Test Auc for H104:  0.6422552722859154 , H118:  0.6170000062045927  , combining together with H143 testset: 0.6347522649999163
#bestValidAuc 0.690075 has a TestAuc of 0.698186 at epoch 6
# Epoch  6  Train_auc : 0.7802418475435675  , Valid_auc :  0.6900751166474599  ,& Test_auc :  0.6981864076708845  Avg Loss:  0.31241230348745985 Train Time (1m 19s) Eval Time (2m 42s)


# RNN 
#bestValidAuc 0.712566 has a TestAuc of 0.730539 at epoch 15
#Test Auc for H104:  0.6530464804197255 , H118:  0.6440123151293581  , combining together with H143 testset: 0.6539902391808247
# Epoch  15  Train_auc : 0.8200194776172784  , Valid_auc :  0.7125661239055848  ,& Test_auc :  0.7305390113122104  Avg Loss:  0.28983430554469425 Train Time (0m 51s) Eval Time (1m 8s)

Epoch  0  Train_auc : 0.7957729127575853  , Valid_auc :  0.7759308710460213  ,& Test_auc :  0.7953852914885522  Avg Loss:  0.3224477071563403 Train Time (0m 52s) Eval Time (1m 4s)


/home/lgindybekhet/.local/lib/python3.5/site-packages/torch/serialization.py:193: UserWarning:

Couldn't retrieve source code for container of type EHR_RNN. It won't be checked for correctness upon loading.



Epoch  1  Train_auc : 0.8209505776747933  , Valid_auc :  0.7952438873124612  ,& Test_auc :  0.8178465436937085  Avg Loss:  0.30958301822344464 Train Time (0m 58s) Eval Time (1m 16s)
Epoch  2  Train_auc : 0.8316153968479696  , Valid_auc :  0.806575315818319  ,& Test_auc :  0.8306146263832282  Avg Loss:  0.30143493572870894 Train Time (0m 51s) Eval Time (1m 18s)
Epoch  3  Train_auc : 0.835963151678813  , Valid_auc :  0.8100120376790829  ,& Test_auc :  0.8274683269257912  Avg Loss:  0.30058792605996126 Train Time (0m 52s) Eval Time (1m 18s)
Epoch  4  Train_auc : 0.825542602961168  , Valid_auc :  0.7939149887409724  ,& Test_auc :  0.8147599367667098  Avg Loss:  0.30079938977956766 Train Time (0m 52s) Eval Time (1m 4s)
Epoch  5  Train_auc : 0.8459661033890722  , Valid_auc :  0.8140522827603278  ,& Test_auc :  0.8339537367931976  Avg Loss:  0.29552135358254117 Train Time (0m 51s) Eval Time (1m 3s)
Epoch  6  Train_auc : 0.8384182172677269  , Valid_auc :  0.8066438917931225  ,& Test_auc :  0.8

In [13]:
best_model.load_state_dict(torch.load(bmodel_st))
best_model.eval()
test_auc_118 = calculate_auc(best_model,test_118,batch_size)
#test_auc_118_sh = calculate_auc(best_model,test_sh_118,batch_size)
#test_auc_118_L = calculate_auc(best_model,test_l_118,batch_size)
#print ('TestAuc Hosp118: ',test_auc_118 , 'pts with less than 5 visits: ',test_auc_118_sh, ' and those with longer history' ,test_auc_118_L )
test_auc_104 = calculate_auc(best_model,test_104,batch_size)
#test_auc_104_sh = calculate_auc(best_model,test_sh_104,batch_size)
#test_auc_104_L = calculate_auc(best_model,test_l_104,batch_size)
#print ('TestAuc Hosp104: ',test_auc_104 , 'pts with less than 5 visits: ',test_auc_104_sh, ' and those with longer history' ,test_auc_104_L )
test_auc_3hosp = calculate_auc(best_model,test_3hosp,batch_size)
#test_auc_3hosp_sh = calculate_auc(best_model,test_sh_3hosp,batch_size)
#test_auc_3hosp_L = calculate_auc(best_model,test_l_3hosp,batch_size)
#print ('TestAuc for 3 hospitals combined: ',test_auc_3hosp , 'pts with less than 5 visits: ',test_auc_3hosp_sh, ' and those with longer history' ,test_auc_3hosp_L )
print ('Test Auc for H104: ',test_auc_104 , ', H118: ',test_auc_118,' , combining together with H143 testset:' ,test_auc_3hosp )

Test Auc for H104:  0.7760716066241817 , H118:  0.7413352136245663  , combining together with H143 testset: 0.7648787425324082


In [14]:
test_sl_v = calculate_auc(best_model,test_sl,batch_size)
test_sl_v

0.8339537367931976

In [15]:
import plotly.plotly as py 
import plotly.graph_objs as go
py.sign_in('LailaRasmy','mzNHzVvwYjcZwBDZx3B7')

train_auc_fg= go.Scatter(x= np.arange(epochs), y=train_auc_allep, name='train')
test_auc_fg= go.Scatter(x= np.arange(epochs), y=test_auc_allep, name='test')
#tests_auc_fg= go.Scatter(x= np.arange(epochs), y=tests_auc_allep, name='test<5')
#testl_auc_fg= go.Scatter(x= np.arange(epochs), y=testl_auc_allep, name='test5+')

valid_auc_fg= go.Scatter(x= np.arange(epochs), y=valid_auc_allep, name='valid')
valid_max = max(valid_auc_allep)
test_max = max(test_auc_allep)
#data = [train_auc_fg,test_auc_fg,valid_auc_fg,tests_auc_fg,testl_auc_fg]#,valid_auc_allep,test_auc_allep] 
data = [train_auc_fg,test_auc_fg,valid_auc_fg]
layout = go.Layout(xaxis=dict(dtick=1))
layout.update(dict(annotations=[go.Annotation(text="Max Valid", x=valid_auc_allep.index(valid_max), y=valid_max)]))
#layout.update(dict(annotations=[go.Annotation(text="Max Test", x=test_auc_allep.index(test_max), y=test_max)]))
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='BiRNN_Auc')
#url = py.plot(data, filename='some-data')  # gen. online plot
#py.image.save_as(data, 'some-data.png') 